When starting a Python project, the first thing we do is import our dependencies.

In [1]:
# Import condynsate. This package is used to simulate, render, and plot dynamic systems
import condynsate

To use condynsate, we first create an instanace of a simulator. The simulator handles simulating dynamic systems, rendering them in real-time, and creating live plots that show us data from the simulation. 

Running this command will both create a simulator and open a visualization window in your default internet browser.

In [2]:
# Create an instance of the simulator with visualization and animation
sim = condynsate.Simulator(visualization=True, # Visualization is the process of rendering your dynamic system in 3D
                           animation=True,     # Animation is the process of live plotting data from the simulation
                           animation_fr=15.    # This parameter is the frame rate of the animator (plots)
                          )

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7003/static/


Next, we load URDF files into the simulator. These URDF files encode visual, collisional, and inertial data of articulated bodies --rigid bodies joined together by joints--. The articulate body format is a common way to define robotic systems.

As we load each URDF file, it will automatically appear in your visualization window.

In [3]:
# Load the wheel
wheel_obj = sim.load_urdf(urdf_path='./wheel_vis/wheel.urdf', # The relative path that leads to the wheel .urdf file 
                          position=[0., 0., 0.],              # The (X, Y, Z) coordinates at which the wheel object is placed           
                          fixed=True,                         # The origin of the wheel (base) is not allowed to move
                          update_vis=True                     # The wheel object will be updated in the visualizer each simulation step
                         )

In [4]:
# Load the target arrow. This arrow will point in the direction that we want to rotate the wheel to.
# When first loaded, the target arrow will not be visible. This is because it is pointing in the same direction 
# as the orange arrow attached to the wheel.
target_obj = sim.load_urdf(urdf_path='./wheel_vis/target_arrow.urdf',
                           position=[0., 0., 0.6655],
                           fixed=True,
                           update_vis=True
                          )

Now we will set up the animator. We define how many suplots we want, how many lines each subplot will have, and style choices for each subplot.

In [5]:
# This defines the first subplot.
# This subplot will plot the current angle of the wheel and the target angle of the wheel.
plot1, lines1 = sim.add_subplot_to_animator(n_lines=2,                 # How many lines are going to be on this subplot
                                            title="Angles vs Time",    # The title of our subplot
                                            x_label="Time [Seconds]",  # The label to give the x axis
                                            y_label="Angles [Rad]",    # The label to give the y axis
                                            colors=["r", "b"],         # The colors of each line
                                            line_widths=[2.5, 2.5],    # The width (thickness) of each line
                                            line_styles=["-", ":"],    # The style of each line (solid and dotted)
                                            labels=["Angle", "Target"] # The name of each line in the subplot
                                           )

In [6]:
# This subplot will track the torque applied to the wheel by the axle motor
plot2, lines2 = sim.add_subplot_to_animator(n_lines=1,
                                            title="Torque vs Time",
                                            x_label="Time [Seconds]",
                                            y_label="Torque [Nm]",
                                            colors=["k"],
                                            line_widths=[2.5]
                                           )

After adding all the subplots that we want, we can now open the animator GUI. This will open a new window that shows the plots that you just defined.

In [7]:
sim.open_animator_gui()

Now we define some functions that let us use the keyboard to change our target angle.

In [8]:
# Set our initial target angle of 0.0 rad.
target_angle = 0.0

# This function uses the simulator to listen for key presses
# and then updates the target angle based on which keys are pressed
def update_target_angle(target_angle):
    
    # Set our new target angle to the same as the previous target angle
    new_target_angle = target_angle
    
    # If the 'a' key is pressed, increase the target angle by a little bit
    if sim.is_pressed('a'):
        new_target_angle = new_target_angle + 0.005*6.2831854

        # Make sure that we don't increase the target angle above pi rads
        if new_target_angle > 3.1415927:
            new_target_angle = 3.1415927

    # If the 'd' key is pressed, decrease the target angle by a little bit
    elif sim.is_pressed('d'):
        new_target_angle = new_target_angle - 0.005*6.2831854

        # Make sure that we don't decrease the target angle below -pi rads
        if new_target_angle < -3.1415927:
            new_target_angle = -3.1415927

    # Return the newly updated target angle
    return new_target_angle

Here we make our controller. The controller will take some values as arguments then use them to calculate how much torque we should apply to the wheel.

In [9]:
def controller(target_angle, angle, angle_vel):
    angle_error = angle - target_angle           # Calculate how far away we are from the target
    torque = -3.0 * angle_error - 2.0*angle_vel  # Calculate what torque we should apply
    return torque

Now we can run our simulation. This is done by calling sim.step() in a while loop. The simulator will automatically handle stepping the dynamics, updating the visualization, and plotting whatever data we send to our two subplots.

To stop the simulation, press 'esc' on your keyboard. To reset the simulation, press 'tab' on your keyboard.

In [10]:
# Run the simulation
while(not sim.is_done):
    ###########################################################################
    # SENSOR
    # Use a sensor to collect the angle of the wheel 
    # and the angular velocity of the wheel
    state = sim.get_joint_state(urdf_obj=wheel_obj,         # The urdf id of the wheel that was generated above
                                joint_name="ground_to_axle" # The name of the joint whose state we want to measure
                               )
    angle = state['position']
    angle_vel = state['velocity']
    ###########################################################################
    # CONTROLLER
    # This is the section where you apply your controller.
    torque = controller(target_angle,
                        angle,
                        angle_vel
                       )
    
    ###########################################################################
    # ACTUATOR
    # Apply the controller calculated torque to the wheel using an actuator.
    sim.set_joint_torque(urdf_obj=wheel_obj,          # The urdf id of the wheel that was generated above
                         joint_name="ground_to_axle", # The name of the joint to which the torque is applied
                         torque=torque,               # The amount of torque we apply
                         show_arrow=True,             # This indicates that we visualize the torque with an arrow
                         arrow_scale=0.3,             # The size of the arrow
                         arrow_offset=0.52            # The amount the arrow is offset from the center of the axle motor  
                        )
    
    ###########################################################################
    # UPDATE THE PLOTS
    # This is how we add data points to the animator
    # Each time step, we note the current time, the angle of the wheel,
    # the target angle of the wheel, and the torque applied.
    # We then send this data to the subplots and lines that we 
    # want them to be plotted on.
    sim.add_subplot_point(subplot_index=plot1,  # The subplot id on which we plot the angle vs time
                          line_index=lines1[0], # The line id to which we send the angle vs time data point
                          x=sim.time,           # The x coordinate of the data point we send (current time)
                          y=angle               # The y coordinate of the data point we send (current angle of the wheel)
                         )
    
    # We then repeat the above procedure for plotting the target angle
    # on the same subplot as the current angle but the second line.
    sim.add_subplot_point(subplot_index=plot1,
                          line_index=lines1[1],
                          x=sim.time,
                          y=target_angle
                         )

    # Finally, we plot the torque vs time on the second subplot.
    sim.add_subplot_point(subplot_index=plot2,
                          line_index=lines2[0],
                          x=sim.time,
                          y=torque
                         )
    
    ###########################################################################
    # UPDATE THE TARGET ANGLE
    # Adjust the target angle by using the code we wrote above
    target_angle = update_target_angle(target_angle)

    # Adjust the target arrow so that it is always pointing in the target angle direction
    sim.set_joint_position(urdf_obj=target_obj,         # The urdf id of the wheel that was generated above
                           joint_name='world_to_arrow', # The name of the joint whose position we want to change
                           position=target_angle,       # The direction we want the target arrow to point in
                           physics=False                # Specifies whether or not to use physics to adjust the position.
                          )
    
    ###########################################################################
    # STEP THE SIMULATION
    sim.step(real_time=True,   # We want our simulation to run in real time (or as close as possible)
              update_vis=True, # We want to update the visualizer each time we step the physics
              update_ani=True  # We want to update the animator each time we step the physics
            )

RESETTING...
Termination command detected. Terminating keyboard listener. Goodbye
